<a href="https://colab.research.google.com/github/EsauHervert/DRP-Machine-Learning/blob/master/Linear_Regression_Multivariate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let us look at the following problem:

$$x = \begin{bmatrix}
x^1\\
x^2
\end{bmatrix}$$

$$D = \begin{bmatrix}
D_1\\
D_2
\end{bmatrix}$$

$$f(x) = D_1x^1+D_2x^2 = D^Tx$$

Now, given some training data ${X,Y}$, how can we reconstruct $f$? We can do it by finding some $w$ such that $w\approx D$. We can use the following to measure how close $w$ is to $D$:

$$\mathcal{L}(X,Y,w) = \frac{1}{2n}\sum_{i=1}^n(w^Tx_i - y_i)^2$$

Using the gradient descent algorithm, we get:

$$\nabla_w\mathcal{L}(X,Y,w) = \frac{1}{n}\sum_{i=1}^n(w^Tx_i-y_i)*x_i = \delta(w)$$

$$w_{j+1} = w_j - \eta \delta(w_j)$$


In [0]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Now, let us look at the following function:

$$f(x) = 2x^1 - 3x^2$$

Note that for these algorithms to converge, we require that the data be features be independent.

In [0]:
X1 = torch.rand(1,200)
X2 = torch.rand(1,200)
Y = 2*X1 - 3*X2
print(Y.size())

torch.Size([1, 200])


In [0]:
Xt = torch.cat((X1, X2), 0)
print(Xt.size())

torch.Size([2, 200])


In [0]:
def GradDescent(X, Y, w, eta, N):
  n,m = X.size()
  for j in range(N):
    wT = torch.transpose(w, 0, 1)
    E = torch.mm(wT, X) - Y
    delta = (E*X).mean(1)
    delta = torch.reshape(delta, (n, 1))
    w -= eta*delta
  return w

In [0]:
w = torch.rand(2,1)
w_new = w.clone()
print(w)

tensor([[0.5491],
        [0.2267]])


In [0]:
def f(w,x):
  wT = torch.transpose(w, 0, 1)
  return torch.mm(wT, x)

In [0]:
def loss(X,Y,w):
  E = f(w,X) - Y
  return .5*(E*E).mean().item()

In [0]:
loss(Xt,Y,w)

0.8937805891036987

In [0]:
def GradDescent(X, Y, w, eta, N):
  n,m = X.size()
  for j in range(N):
    E = f(w,X) - Y
    delta = (E*X).mean(1)
    delta = torch.reshape(delta, (n, 1))
    w -= eta*delta
  return w

In [0]:
w_new = GradDescent(Xt, Y, w_new, .01, 1000)
print(w_new)
print(loss(Xt,Y, w_new))

tensor([[ 1.0001],
        [-1.9619]])
0.08688648045063019


In [0]:
w_new = GradDescent(Xt, Y, w_new, .01, 10000)
print(w_new)
print(loss(Xt,Y, w_new))

tensor([[ 1.9997],
        [-2.9997]])
5.8201861108386765e-09


Now let us look at the function

$$f(x) = x^1 - 5x^2 + 3x^3 +4x^4$$

and see if our method works for this function as well.

In [0]:
X1 = torch.rand(1,200)
X2 = torch.rand(1,200)
X3 = torch.rand(1,200)
X4 = torch.rand(1,200)
Y = X1 - 5*X2 + 3*X3 + 4*X4
print(Y.size())

Xt = torch.cat((X1, X2, X3, X4), 0)
print(Xt.size())

w = torch.rand(4,1)
w_new = w.clone()
print(w)

loss(Xt,Y,w)

w_new = GradDescent(Xt, Y, w_new, .01, 10000)
print(w_new)
print(loss(Xt,Y, w_new))

torch.Size([1, 200])
torch.Size([4, 200])
tensor([[0.2767],
        [0.3214],
        [0.0051],
        [0.2987]])
tensor([[ 0.9983],
        [-4.9962],
        [ 3.0012],
        [ 3.9968]])
9.916702765622176e-07


Now that we have the linear case, we can look at the quadratic case:

$$f(x) = y_0 + D^Tx + x^TAx$$

$$D = \begin{bmatrix}
d_1\\
d_2\\
\vdots\\
d_k
\end{bmatrix}$$

$$A = \begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1k}\\
a_{21} & a_{22} & \cdots & a_{2k}\\
\vdots & \vdots & \ddots &\vdots\\
a_{k1} & a_{k2} & \cdots & a_{1k}\\
\end{bmatrix}$$

$$f(x) = y_0 + d_1x^1 + d_2x^2+\cdots + d_kx^k + a_{11}x^1x^1 + a_{12}x^1x^2 + \cdots a_{21}x^2x^1 + \cdots + a_{kk}x^kx^k$$

We will assume that $A$ is symmetric, i.e. $a_{ij} = a_{ji}$. given that $x^ix^j = x^jx^i$, we get:

$$f(x) = d_1x^1 + d_2x^2+\cdots + d_kx^k + a_{11}x^1x^1 + 2a_{12}x^1x^2 + \cdots +a_{ii}x^ix^i + 2a_ia_jx^ix^j + \cdots + a_{kk}x^kx^k$$

Like before we will look for a "$w$" that will minimize the MSE error of the data with our guess, The difference is that this time we will have a $w_0$, $D_w$,s and a $A_w$, i.e.

$$f_w(x) = w_0 + D_w^Tx + x^TA_wx$$

Defining a parameter $w$ to contain all the information about $w_0$, $D_w$, and $A_w$:

$$w = \begin{bmatrix}
w_0\\
D_w\\
A_w
\end{bmatrix}$$

Now, the MSE loss has the form:

$$\mathcal{L}(X,Y,w) = \frac{1}{2n}\sum_{i=1}^n \left(w_0 + D_w^Tx_i + x_i^TA_wx_i -y_i\right)^2$$

We can define the gradient w.r.t. $w$ as:

$$\nabla_w = \begin{bmatrix}
\nabla_{w_0}\\
\nabla_{D_w}\\
\nabla_{A_w}
\end{bmatrix}$$

Now note the following:

$$\nabla_w w = 1$$
$$\nabla_w w^Tx = x$$
$$\nabla_W x^TWx = xx^T$$

So we have that using the above and the fact that 

\begin{align*}
  \nabla_w \mathcal{L}(X,Y,w) &= \begin{bmatrix}
\nabla_{w_0}\mathcal{L}(X,Y,w)\\
\nabla_{D_w}\mathcal{L}(X,Y,w)\\
\nabla_{A_w}\mathcal{L}(X,Y,w)
\end{bmatrix}\\\\
  &=  \begin{bmatrix}
\frac{1}{n}\sum_{i=1}^n\left(w_0 + D_w^Tx_i + x_i^TA_wx_i - y_i\right)\\
\frac{1}{n}\sum_{i=1}^n\left(w_0 + D_w^Tx_i + x_i^TA_wx_i - y_i\right)*x_i\\
\frac{1}{n}\sum_{i=1}^n\left(w_0 + D_w^Tx_i + x_i^TA_wx_i - y_i\right)*x_ix_i^T
\end{bmatrix}\\\\
  &= \frac{1}{n}\sum_{i=1}^n\left(w_0 + D_w^Tx_i + x_i^TA_wx_i - y_i\right)\begin{bmatrix}
1\\
x_i\\
x_ix_i^T
\end{bmatrix}
\end{align*}

Defining the error as:
$$e_i(w) = w_0 + D_w^Tx_i + x_i^TA_wx_i - y_i$$

we can define two deltas:

$$\delta_0(w) = \frac{1}{n}\sum_{i=1}^ne_i(w)$$

$$\delta_1(w) = \frac{1}{n}\sum_{i=1}^ne_i(w)*x_i$$

$$\delta_2(w) = \frac{1}{n}\sum_{i=1}^ne_i(w)*x_ix_i^T$$

Thus we can define our update rules as follows:

$$w_o(j+1) = w_0(j) - \eta \delta_0(w_j)$$

$$D_w(j+1) = D_w(j) - \eta \delta_1(w_j)$$

$$A_w(j+1) = A_w(j) - \eta \delta_2(w_j)$$

Note that since $xx^T$ is symmetric, we have that the update will return a symmetric matrix for $A_w$ as long as $A_w(1)$ is symmetrix.

Now, let us look at the following problem on the interval $[-1,1]^2$:

$$f(x) = 5 + 2x^1 + 3x^2 + x^1x^1 + 4x^1x^2 + x^2x^2$$

$$y_0 = 5$$

$$D = \begin{bmatrix}
2\\
3
\end{bmatrix}$$

$$A = \begin{bmatrix}
1 & 2\\
2 & 1
\end{bmatrix}$$

In [0]:
X1 = 2*torch.rand(1,200) - 1
X2 = 2*torch.rand(1,200) - 1
Y = 5 + 2*X1 + 3*X2 + X1*X1 + 4*X1*X2 + X2*X2
print(Y.size())

torch.Size([1, 200])


In [0]:
Xt = torch.cat((X1,X2), 0)
print(Xt.size())

torch.Size([2, 200])


We will define a diag function since we only care of the $x_i^TA_wX_i$ terms in the matrix $X^TAX$ matrix.

In [0]:
def diag(A):
  n,m = A.size()
  diag = torch.zeros(1,n)

  for i in range(n):
    diag[0, i] = A[i,i]
  
  return diag

Now defining the function that given some data $X = \begin{bmatrix}
x_1 \cdots x_n\end{bmatrix}$ and some weights $w_0$, $D_w$, and $A_w$, we get

$$f(x;w_0, D_w, A_w) = w_0 + D_w^Tx + x^TA_wx$$

In [0]:
def f(X, w0, Dw, Aw):
  DwT = torch.transpose(Dw, 0, 1)
  DwTX = torch.mm(DwT, X)
  XT = torch.transpose(X, 0, 1)
  XTAw = torch.mm(XT, Aw)
  XTAwX = torch.mm(XTAw, X)
  
  return w0 + DwTX + diag(XTAwX)

Now, we can generate random tensors, but we need a symmetric matrix so we need to create a function that will turn a random matrix to a symmetric matrix.

In [0]:
def symm(A):
  n,m = A.size()
  for i in range(n):
    for j in range(n - i - 1):
      k = n - j - 1
      A[i, k] = A[k, i]
  
  return A

In [0]:
w0 = torch.rand(1,1)
Dw = torch.rand(2,1)
Aw = torch.rand(2,2)
Aw = symm(Aw)

print(w0)
print(Dw)
print(Aw)

tensor([[0.8250]])
tensor([[0.1653],
        [0.5387]])
tensor([[0.1283, 0.2407],
        [0.2407, 0.7997]])


In [0]:
print(f(Xt, w0, Dw, Aw).size())

torch.Size([1, 200])


Now, defining a loss function for all three of the weight sets:

In [0]:
def loss(X, Y, w0, Dw, Aw):
  E = f(X, w0, Dw, Aw) - Y
  return .5*(E*E).mean().item()

In [0]:
loss(Xt, Y, w0, Dw, Aw)

12.208385467529297

Now, we will use the class Weights so that we can return multiple set of weights in the gradient descent algorithm.

In [0]:
class Weights():
    def __init__(self, scalar, vector, matrix):
        self.scalar = scalar
        self.vector = vector
        self.matrix = matrix

Now here is the gradient descent rule that we derived.

In [0]:
def GradDescent(X, Y, w0, Dw, Aw, eta, N):
  m,n = X.size()

  # This will hold all our weights
  W = Weights(w0, Dw, Aw)

  for j in range(N):

    # We calculate the function values given the current weights
    F = f(Xt, w0, Dw, Aw)

    # This is the delta values that we will use to update the weights
    delta0 = torch.zeros(1,1)
    delta1 = torch.zeros(m,1)
    delta2 = torch.zeros(m,m)

    # To calculate the delta values we will loop through the samples for simplicity sake
    for i in range(n):
      xi = torch.reshape(Xt[:,i], (m, 1))
      yi = Y[:,i]
      fi = F[:,i]
      ei = fi - yi

      delta0 += ei
      delta1 += ei*xi
      delta2 += ei*torch.mm(xi, torch.transpose(xi, 0, 1))
    
    # Average
    delta0 = (1/n)*delta0
    delta1 = (1/n)*delta1
    delta2 = (1/n)*delta2

    # Delta update rule
    W.scalar -= eta*delta0
    W.vector -= eta*delta1
    W.matrix -= eta*delta2


  return W

Now, running the algorithm we can see that this algorithm does in fact converge close to the true solution.

In [0]:
Weights = GradDescent(Xt, Y, w0, Dw, Aw, .01, 5000)

In [0]:
w0f = Weights.scalar
Dwf = Weights.vector
Awf = Weights.matrix

print(w0f)
print(Dwf)
print(Awf)

tensor([[4.9806]])
tensor([[2.0013],
        [3.0006]])
tensor([[1.0202, 2.0005],
        [2.0005, 1.0327]])


In [0]:
loss(Xt, Y, w0f, Dwf, Awf)

6.125760410213843e-05